
# Source localization with single dipole fit


The aim of this lecture is to show you how to do sequential and
fixed dipole fitting with MNE-Python.

In [1]:
%matplotlib qt
import matplotlib.pyplot as plt

import os
import numpy as np
import mne

mne.set_log_level('WARNING')

# Change the following path to where the folder ds000117-practical is on your disk
data_path = os.path.expanduser("~/work/data/ds000117-practical/")

raw_fname = os.path.join(data_path,
    'derivatives/meg_derivatives/sub-01/ses-meg/meg/sub-01_ses-meg_task-facerecognition_run-01_proc-sss_meg.fif')

epochs_fname = raw_fname.replace('_meg.fif', '-epo.fif')

In [2]:
subjects_dir = os.path.join(data_path, 'derivatives/freesurfer-reconall/')
subject = 'sub-01'
trans_fname = os.path.join(data_path,
    'derivatives/meg_derivatives/sub-01/ses-meg/meg/sub-01-trans.fif')
bem_fname = os.path.join(data_path,
    'derivatives/meg_derivatives/sub-01/ses-meg/meg/sub-01-bem.fif')
fname_surf_lh = os.path.join(subjects_dir, subject, 'surf', 'lh.white')

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
      <li>Check that the head geometry and sensor alignment is correct</li>
    </ul>
</div>

You will need to use the `mne.viz.plot_alignment` function that can take as input a `bem` parameter of type `ConductorModel`

In [3]:
mne.viz.plot_alignment?

In [4]:
bem = mne.bem.read_bem_solution(bem_fname)
bem

<ConductorModel  |  BEM (1 layer)>

In [5]:
# TODO
info = mne.io.read_info(epochs_fname)
fig = mne.viz.plot_alignment(info, trans_fname, subject=subject, dig=True,
                             subjects_dir=subjects_dir, bem=bem, verbose=True);

Could not find the surface for head in the provided BEM model, looking in the subject directory.
Using outer_skin.surf for head surface.
Getting helmet for system 306m


## Let's localize the N170m (using MEG only) using dipole fitting

In [6]:
epochs = mne.read_epochs(epochs_fname)

In [7]:
epochs.pick_types(meg=True, eeg=False)

<EpochsFIF  |   79 events (all good), -0.2 - 2 sec, baseline [-0.2, 0], ~129.1 MB, data loaded,
 'face/famous/first': 13
 'face/famous/immediate': 3
 'face/famous/long': 6
 'face/unfamiliar/first': 17
 'face/unfamiliar/immediate': 4
 'face/unfamiliar/long': 6
 'scrambled/first': 15
 'scrambled/immediate': 9
 'scrambled/long': 6>

In [8]:
cov = mne.compute_covariance(epochs, rank='info')

In [9]:
evoked_face = epochs['face'].average()
evoked_scrambled = epochs['scrambled'].average()

In [10]:
contrast = mne.combine_evoked([evoked_face, evoked_scrambled], [0.5, -0.5])
contrast.crop(None, 0.2)
contrast.plot();

In [11]:
# Fit a dipole using a sequential (time-varying position and orientation) fit
contrast_crop = contrast.copy().crop(0.175, 0.190)
dip, residual = mne.fit_dipole(contrast_crop, cov, bem_fname,
                               trans_fname)
print(dip)

<Dipole  |  n_times : 6, tmin : 0.173, tmax : 0.190>


In [12]:
# Look at our result
print(dip.gof)

[33.67613685 36.70061657 35.58746073 34.71198179 38.51052585 41.45988139]


In [13]:
dip.plot_locations(subject=subject, trans=trans_fname,
                   subjects_dir=subjects_dir, mode='orthoview');

<div class="alert alert-success">
    <b>EXERCISE</b>:
     <ul>
      <li>Try to improve the dipole fit by using a subselection of channels [0] </li>
      <li>What is the improvement in max GOF from using a subselection of channels?</li>     
      <li>Should you also try to maximize GOF?</li>
    </ul>
</div>

Tips and tricks:

   - [0] Subselect channels (think about ``mne.read_selection(...)``, ``evoked.pick_channels(...)``)

In [14]:
# TODO
selection = mne.read_selection('Left', info=contrast.info)
# Fit a dipole using a sequential (time-varying position and orientation) fit
dip, residual = \
    mne.fit_dipole(contrast_crop.copy().pick_channels(selection),
                   cov, bem_fname, trans_fname)
print(dip)

<Dipole  |  n_times : 6, tmin : 0.173, tmax : 0.190>


In [15]:
print(dip.gof)

[38.15096387 20.45112159 47.88457627 48.48499646 19.37190885 48.05207802]


In [16]:
dip.plot_locations(subject=subject, trans=trans_fname,
                   subjects_dir=subjects_dir, mode='orthoview');